In [1]:
import cv2
import argparse
import numpy as np
import utils
import os 
import time

In [2]:
%%time
utils.extract_frames("/data/perso/ObamaWalk.mp4", out_dir='/data/perso/out/video_to_image', save=True, skip=1)

Frames saved to `/data/perso/out/video_to_image/` directory.
CPU times: user 2min 13s, sys: 19.1 s, total: 2min 33s
Wall time: 5min 12s


In [3]:
%%time
# read input image
list_images = utils.get_files_by_ext("/data/perso/out/video_to_image", "jpg")
print("We got ", len(list_images), " images")

We got  4479  images
CPU times: user 359 ms, sys: 315 ms, total: 674 ms
Wall time: 5.06 s


In [4]:
%%time
classes_file = "object-detection-opencv/yolov3.txt"
weights_file = "object-detection-opencv/yolov3.weights"
conf_file = "object-detection-opencv/yolov3.cfg"

conf_threshold = 0.5
nms_threshold = 0.4

# for image_path in list_images:
image = cv2.imread(list_images[0])

Width = image.shape[1]
Height = image.shape[0]

scale = 0.00392

# read class names from text file
classes = None
with open(classes_file, 'r') as f:
    classes = [line.strip() for line in f.readlines()]

# generate different colors for different classes 
COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

# read pre-trained model and config file
# reads the weights and config file and creates the network.
net = cv2.dnn.readNet(weights_file, conf_file)

batch_size = 10
batch_images = list()

for i in range(len(list_images)):
# for i in range(10, 20):
    batch_images.append(list_images[i])
    if i == len(list_images)-1 or len(batch_images)%batch_size==0:      
        
        time_min_blobing = time.time()
        # create input blob 
        # prepares the input image to run through the deep neural network
        blob = utils.image_to_matrix(batch_images)

        # set input blob for the network
        net.setInput(blob)
        time_max_blobing = time.time()
        elapsed_time_blobing = time_max_blobing - time_min_blobing
        print("The blobing phase took ", round(elapsed_time_blobing,2), "s to be executed")
        print("\n")
        
        
        time_min_detection = time.time()
        # run inference through the network
        # and gather predictions from output layers
        outs = net.forward(utils.get_output_layers(net))

        print(len(outs))
        for out in outs:
            print(out.shape)
        
        # identify objects into the list of images analyzed by the network
        dict_obj_detected = {}
        dict_obj_detected = utils.detect_object(outs, batch_images, Width, Height)

        # identify pictures with at least one perso
        dict_danger = utils.detect_danger(dict_obj_detected)
        for image_name, is_dangerous in dict_danger.items():
            print("Is", image_name, "dangerous ? \t\t\t" , is_dangerous)
        time_max_detection = time.time()
        elapsed_time_detection = time_max_detection - time_min_detection
        print("The detection took ", round(elapsed_time_detection,2), "s to be executed")
        
        time_min_draw_bbox = time.time()
        # Draw bounding boxes and save images
        for image_path, image_items in dict_obj_detected.items():
            utils.get_bounding_box(image_path, image_items, \
                                   classes, COLORS, conf_threshold, nms_threshold
                                  )
        time_max_draw_bbox = time.time()
        elapsed_time_draw_bbox = time_max_draw_bbox - time_min_detection
        print("The draw of the bounding boxes and the image save took ", round(elapsed_time_draw_bbox,2), "s to be executed")         
        print("---------------------------------------------------------------------")
        
        batch_images = list()
    


The blobing phase took  0.3 s to be executed


3
(10, 507, 85)
(10, 2028, 85)
(10, 8112, 85)
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00000.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00001.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00002.jpg dangerous ? 			 False
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00003.jpg dangerous ? 			 False
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00004.jpg dangerous ? 			 False
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00005.jpg dangerous ? 			 False
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00006.jpg dangerous ? 			 False
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00007.jpg dangerous ? 			 False
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00008.jpg dangerous ? 			 False
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00009.jpg dangerous ? 			 False
The detection took  15.11 s to be executed
The draw of the 

The draw of the bounding boxes and the image save took  12.66 s to be executed
---------------------------------------------------------------------
The blobing phase took  0.24 s to be executed


3
(10, 507, 85)
(10, 2028, 85)
(10, 8112, 85)
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00080.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00081.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00082.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00083.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00084.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00085.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00086.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00087.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00088.jpg dangerous ? 			 True
I

The draw of the bounding boxes and the image save took  13.21 s to be executed
---------------------------------------------------------------------
The blobing phase took  0.21 s to be executed


3
(10, 507, 85)
(10, 2028, 85)
(10, 8112, 85)
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00160.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00161.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00162.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00163.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00164.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00165.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00166.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00167.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00168.jpg dangerous ? 			 True
I

The draw of the bounding boxes and the image save took  12.98 s to be executed
---------------------------------------------------------------------
The blobing phase took  0.26 s to be executed


3
(10, 507, 85)
(10, 2028, 85)
(10, 8112, 85)
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00240.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00241.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00242.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00243.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00244.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00245.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00246.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00247.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00248.jpg dangerous ? 			 True
I

The draw of the bounding boxes and the image save took  12.67 s to be executed
---------------------------------------------------------------------
The blobing phase took  0.22 s to be executed


3
(10, 507, 85)
(10, 2028, 85)
(10, 8112, 85)
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00320.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00321.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00322.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00323.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00324.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00325.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00326.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00327.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame00328.jpg dangerous ? 			 True
I

...

The draw of the bounding boxes and the image save took  17.5 s to be executed
---------------------------------------------------------------------
The blobing phase took  0.53 s to be executed


3
(10, 507, 85)
(10, 2028, 85)
(10, 8112, 85)
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame04160.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame04161.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame04162.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame04163.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame04164.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame04165.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame04166.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame04167.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame04168.jpg dangerous ? 			 True
Is

The draw of the bounding boxes and the image save took  17.08 s to be executed
---------------------------------------------------------------------
The blobing phase took  0.41 s to be executed


3
(10, 507, 85)
(10, 2028, 85)
(10, 8112, 85)
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame04240.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame04241.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame04242.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame04243.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame04244.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame04245.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame04246.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame04247.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/ObamaWalk.mp4_frame04248.jpg dangerous ? 			 True
I

Is /data/perso/out/video_to_image/output_with_bounding_box/ObamaWalk.mp4_frame00100.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/output_with_bounding_box/ObamaWalk.mp4_frame00125.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/output_with_bounding_box/ObamaWalk.mp4_frame00150.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/output_with_bounding_box/ObamaWalk.mp4_frame00175.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/output_with_bounding_box/ObamaWalk.mp4_frame00200.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/output_with_bounding_box/ObamaWalk.mp4_frame00225.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/output_with_bounding_box/ObamaWalk.mp4_frame00250.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/output_with_bounding_box/ObamaWalk.mp4_frame00275.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/output_with_bounding_box/ObamaWalk.mp4_frame00300.jpg dangerous ? 			 True
Is /data/perso/out/

3
(10, 507, 85)
(10, 2028, 85)
(10, 8112, 85)
Is /data/perso/out/video_to_image/output_with_bounding_box/ObamaWalk.mp4_frame01600.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/output_with_bounding_box/ObamaWalk.mp4_frame01625.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/output_with_bounding_box/ObamaWalk.mp4_frame01650.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/output_with_bounding_box/ObamaWalk.mp4_frame01675.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/output_with_bounding_box/ObamaWalk.mp4_frame01700.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/output_with_bounding_box/ObamaWalk.mp4_frame01725.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/output_with_bounding_box/ObamaWalk.mp4_frame01750.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/output_with_bounding_box/ObamaWalk.mp4_frame01775.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/output_with_bounding_box/ObamaWalk.mp4_frame0180

The blobing phase took  0.33 s to be executed


3
(10, 507, 85)
(10, 2028, 85)
(10, 8112, 85)
Is /data/perso/out/video_to_image/output_with_bounding_box/ObamaWalk.mp4_frame03100.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/output_with_bounding_box/ObamaWalk.mp4_frame03125.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/output_with_bounding_box/ObamaWalk.mp4_frame03150.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/output_with_bounding_box/ObamaWalk.mp4_frame03175.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/output_with_bounding_box/ObamaWalk.mp4_frame03200.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/output_with_bounding_box/ObamaWalk.mp4_frame03225.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/output_with_bounding_box/ObamaWalk.mp4_frame03250.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/output_with_bounding_box/ObamaWalk.mp4_frame03275.jpg dangerous ? 			 True
Is /data/perso/out/video_to_image/

In [5]:
utils.convert_frames_to_video("/data/perso/out/video_to_image/output_with_bounding_box/", \
                              "/data/perso/out/output_video/ObamaWalkWithBB.mp4", \
                              24, (Width,Height))

100%|██████████| 4306/4306 [05:43<00:00, 12.53it/s]
